In [8]:
#Import Libraries
import cv2
from ultralytics import YOLO
import tkinter as tk
from tkinter import Label, Button
from PIL import Image, ImageTk

In [9]:
#Filepath of the trained model
model_path = 'trained_models/yolov5_screwtype_trained.pt'

# Load the trained model for real-time detection
model = YOLO(model_path)

# Get class names from the model
class_names = model.names

In [10]:
# Initialize Tkinter window
root = tk.Tk()
root.title("Screw Real-Time Component Detection")

# Create label for video feed
video_label = Label(root)
video_label.pack()

# Create label for detection details
details_label = Label(root, text="Detected Objects: ", font=("Arial", 12), justify=tk.LEFT)
details_label.pack()

# Frame to display cropped object images
objects_frame = tk.Frame(root)
objects_frame.pack()

In [11]:
# Open webcam (but don't start capturing yet)
cap = None 
running = False # Flag to control detection loop 

def start_detection():
    global cap, running 
    if not running:
        cap = cv2.VideoCapture(0) # Open webcam
        running = True # Set up video capture
        video_label.config(text="Starting detection...") 
        detect_objects()

def stop_detection():
    global cap, running
    if running:
        running = False # Stop detection loop
        cap.release() # Release webcam
        cv2.destroyAllWindows() # Close all OpenCV windows
        video_label.config(image='') # Clear video label
        details_label.config(text="Detection Stopped") 

def detect_objects():
    global running
    if not running:
        return

    ret, frame = cap.read()
    if not ret:
        return

    # Run object detection
    results = model(frame)
    detected_text = "Detected Objects:\n"

    # Clear previous object thumbnails
    for widget in objects_frame.winfo_children():
        widget.destroy()

    best_detections = {} # Store best detections
    best_thumbnails = {} # Store best detections and thumbnails


    for result in results:
        for box in result.boxes:
            conf = box.conf[0].item()
            if conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = int(box.cls[0].item())
                class_name = class_names.get(cls, f"Class {cls}")

                # if class_name not in best_detections or conf > best_detections[class_name][0]:
                #     cropped = frame[y1:y2, x1:x2]
                #     cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
                #     cropped_img = Image.fromarray(cropped)
                #     cropped_img.thumbnail((100, 100))
                #     best_detections[class_name] = (conf, cropped_img)

                # # If this is a better detection, update UI
                # if (class_name not in best_thumbnails) or (conf > best_thumbnails[class_name][3]):
                #     # Convert to Tkinter image
                #     cropped = frame[y1:y2, x1:x2]
                #     cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
                #     cropped_img = Image.fromarray(cropped)
                #     cropped_img.thumbnail((100, 100))
                #     cropped_imgtk = ImageTk.PhotoImage(cropped_img)

                #     # Create or update frame
                #     if class_name not in best_thumbnails:
                #         item_frame = tk.Frame(objects_frame)
                #         item_frame.pack(pady=5)

                #         img_label = tk.Label(item_frame, image=cropped_imgtk)
                #         img_label.image = cropped_imgtk
                #         img_label.pack()

                #         txt_label = tk.Label(item_frame, text=f"{class_name}\\nConf: {conf:.2f}", font=("Arial", 10))
                #         txt_label.pack()

                #         best_thumbnails[class_name] = (item_frame, img_label, txt_label, conf)

                #     else:
                #         item_frame, img_label, txt_label, _ = best_thumbnails[class_name]
                #         img_label.configure(image=cropped_imgtk)
                #         img_label.image = cropped_imgtk
                #         txt_label.config(text=f"{class_name}\\nConf: {conf:.2f}")
                #         best_thumbnails[class_name] = (item_frame, img_label, txt_label, conf)


                #     for class_name, (conf, cropped_img) in best_detections.items():
                #         cropped_imgtk = ImageTk.PhotoImage(cropped_img)

                #         obj_frame = tk.Frame(objects_frame)
                #         obj_frame.pack(side=tk.LEFT, padx=5)

                #         img_label = tk.Label(obj_frame, image=cropped_imgtk)
                #         img_label.image = cropped_imgtk
                #         img_label.pack()

                #         text_label = tk.Label(obj_frame, text=f"{class_name}\nConf: {conf:.2f}", font=("Arial", 10))
                #         text_label.pack()

                #         detected_text += f"{class_name}: {conf:.2f}\n"

                label = f"{class_name}: {conf:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)



    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = Image.fromarray(frame)
    frame.thumbnail((600, 400))
    imgtk = ImageTk.PhotoImage(frame)

    video_label.config(image=imgtk)
    video_label.image = imgtk

    details_label.config(text=detected_text)

    video_label.after(10, detect_objects)

In [12]:
# Create start and stop buttons
start_button = Button(root, text="Start Detection", command=start_detection, font=("Arial", 12)) 
start_button.pack(pady=5) 

stop_button = Button(root, text="Stop Detection", command=stop_detection, font=("Arial", 12))
stop_button.pack(pady=5)

# Start Tkinter loop
root.mainloop()



0: 480x640 (no detections), 18.9ms
Speed: 2.2ms preprocess, 18.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 Flat head socket screws, 16.0ms
Speed: 2.6ms preprocess, 16.0ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 Flat head socket screws, 16.2ms
Speed: 2.8ms preprocess, 16.2ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Flat head socket screw, 16.1ms
Speed: 2.2ms preprocess, 16.1ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 Flat head socket screws, 16.1ms
Speed: 2.3ms preprocess, 16.1ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 Flat head socket screws, 16.1ms
Speed: 2.3ms preprocess, 16.1ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 Flat head socket screws, 16.2ms
Speed: 2.5ms preprocess, 16.2ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x64

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\gauta\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\gauta\AppData\Local\Temp\ipykernel_26904\3670986522.py", line 18, in stop_detection
    cv2.destroyAllWindows() # Close all OpenCV windows
    ^^^^^^^^^^^^^^^^^^^^^^^
cv2.error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'

